<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/Bot_Funcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install ta
!pip install yfinance
!pip install xgboost

In [27]:
import yfinance as yf
import numpy as np
import pandas as pd
import ta
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
XGBClassifier(
    n_estimators=200,
    max_depth=4,                 # menor profundidade = menos overfitting
    subsample=0.8,               # usa 80% dos dados por árvore
    colsample_bytree=0.8,        # usa 80% das features por árvore
    learning_rate=0.05,          # suaviza o aprendizado
    early_stopping_rounds=10,    # para de treinar se não melhorar
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# 1. Configurações
ASSETS = ASSETS = ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "XRP-USD",
          "AVAX-USD", "AAVE-USD", "DOT-USD", "NEAR-USD", "ADA-USD"]

TIMEFRAMES = [
    {"interval": "15m", "period": "30d", "atr": 0.02},
    {"interval": "1h", "period": "90d", "atr": 0.03},
    {"interval": "1d", "period": "1000d", "atr": 0.05}
]
TELEGRAM_TOKEN = "8044593190:AAFtUWYHd3uqd-AtQi3uqg42F9G6uV95v8k"
TELEGRAM_CHAT_ID = "-4744645054"

# 2. Obtenção de dados

def get_stock_data(asset, interval="15m", period="700d"):
    data = yf.download(asset, period=period, interval=interval, progress=False, auto_adjust=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    data.columns = [col.split()[-1] if " " in col else col for col in data.columns]
    # Remove colunas duplicadas ou com sufixos como 'Adj Close'
    data = data.loc[:, ~data.columns.duplicated()]
    col_map = {}
    for col in data.columns:
        for std_col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"]:
            if std_col.lower() in col.lower():
                col_map[col] = std_col
    data = data.rename(columns=col_map)
    # Garante que apenas as colunas necessárias estejam presentes
    data = data[["Open", "High", "Low", "Close", "Volume"]]
    required = ["Open", "High", "Low", "Close", "Volume"]
    if not all(col in data.columns for col in required):
        raise ValueError(f"⚠️ Dados de {asset} não possuem todas as colunas necessárias.")
    return data

# 3. Indicadores

def calculate_indicators(data):
    data = data.copy()
    data.reset_index(drop=True, inplace=True)

    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if isinstance(data[col], pd.DataFrame):
            data[col] = data[col].iloc[:, 0]
        elif isinstance(data[col].iloc[0], (list, np.ndarray)):
            data[col] = data[col].apply(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        data[col] = data[col].astype(float)

    # Indicadores
    data["RSI"] = ta.momentum.RSIIndicator(close=data["Close"], window=14).rsi()
    data["SMA_50"] = ta.trend.SMAIndicator(close=data["Close"], window=50).sma_indicator()
    data["SMA_200"] = ta.trend.SMAIndicator(close=data["Close"], window=200).sma_indicator()

    macd = ta.trend.MACD(close=data["Close"])
    data["MACD"] = macd.macd()
    data["MACD_Signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close=data["Close"], window=20)
    data["Bollinger_Upper"] = bb.bollinger_hband()
    data["Bollinger_Lower"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["ADX"] = adx.adx()

    stoch = ta.momentum.StochasticOscillator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["Stoch_K"] = stoch.stoch()
    data["Stoch_D"] = stoch.stoch_signal()

    data["TP"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["VWAP"] = (data["TP"] * data["Volume"]).cumsum() / (data["Volume"].replace(0, np.nan).cumsum())
    data.drop("TP", axis=1, inplace=True)

    data["Doji"] = ((abs(data["Close"] - data["Open"]) / (data["High"] - data["Low"] + 1e-9)) < 0.1).astype(int)
    data["Engulfing"] = ((data["Open"].shift(1) > data["Close"].shift(1)) & (data["Open"] < data["Close"]) &
                          (data["Close"] > data["Open"].shift(1)) & (data["Open"] < data["Close"].shift(1))).astype(int)
    data["Hammer"] = (((data["High"] - data["Low"]) > 3 * abs(data["Open"] - data["Close"])) &
                       ((data["Close"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6) &
                       ((data["Open"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6)).astype(int)

    data.dropna(inplace=True)
    return data



# 4. Modelo

# Substituição da função train_ml_model para usar XGBoost no lugar de RandomForestClassifier

from xgboost import XGBClassifier

def train_ml_model(data, verbose=False):
    if len(data) < 100:
        return None

    df = data.copy()

    # Adiciona previsão do LSTM como feature (stacking)
    try:
        lstm_model = train_lstm_model(df)
        lstm_preds = []
        for i in range(len(df)):
            sub_df = df.iloc[:i+1]
            if len(sub_df) < 20:
                lstm_preds.append(np.nan)
            else:
                pred = predict_with_lstm(lstm_model, sub_df)
                lstm_preds.append(pred)
        df["LSTM_PRED"] = lstm_preds
    except Exception as e:
        print(f"⚠️ Erro ao gerar LSTM_PRED: {e}")
        df["LSTM_PRED"] = np.nan

    # Sinal com base em retorno futuro
    df["Future_Close"] = df["Close"].shift(-5)
    df["Future_Return"] = df["Future_Close"] / df["Close"] - 1
    df["Signal"] = np.select([
        df["Future_Return"] > 0.015,
        df["Future_Return"] < -0.015
    ], [1, -1], default=0)

    features = get_feature_columns()
    df.dropna(inplace=True)
    X = df[features]
    y = df["Signal"].map({-1: 0, 0: 1, 1: 2})  # Re-mapeamento para o XGBoost

    signal_count = (y == 1).sum()
    if verbose:
        print(f"📉 Sinais de COMPRA encontrados: {signal_count} em {len(y)} candles")

    if len(np.unique(y)) < 2:
        return None

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    if len(np.unique(y_train)) < 2:
        return None

    model = XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric="mlogloss",
        random_state=42
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    report = classification_report(y_val, y_pred, output_dict=True, zero_division=0)

    model.validation_score = {
        "accuracy": report.get("accuracy"),
        "precision": report.get("1", {}).get("precision"),
        "recall": report.get("1", {}).get("recall"),
        "f1": report.get("1", {}).get("f1-score")
    }

    feature_importances = dict(zip(features, model.feature_importances_))
    sorted_feat = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)
    print("\n📊 Importância das features (XGBoost):")
    for feat, score in sorted_feat[:5]:
        print(f"   {feat}: {score:.4f}")

    data["LSTM_PRED"] = df["LSTM_PRED"]  # Garante acesso à coluna no run_analysis
    return model

    # 4.1 MODELO LSTM
def prepare_lstm_data(data, feature_col="Close", window_size=20):
    df = data.copy().reset_index(drop=True)
    values = df[feature_col].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(values)

    X, y = [], []
    for i in range(window_size, len(scaled)):
        X.append(scaled[i - window_size:i, 0])
        y.append(scaled[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X, y, scaler

# Novo: treina o modelo LSTM

def train_lstm_model(data, window_size=20, verbose=False):
    if len(data) < window_size + 20:
        return None

    X, y, scaler = prepare_lstm_data(data, window_size=window_size)

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=10, batch_size=32, verbose=0 if not verbose else 1)

    model.scaler = scaler
    model.window_size = window_size
    return model

# Novo: previsão com LSTM

def predict_with_lstm(model, data):
    df = data.copy().reset_index(drop=True)
    values = df["Close"].values.reshape(-1, 1)
    scaled = model.scaler.transform(values)

    last_sequence = scaled[-model.window_size:]
    X_pred = np.reshape(last_sequence, (1, model.window_size, 1))
    predicted_scaled = model.predict(X_pred)[0][0]
    predicted_price = model.scaler.inverse_transform([[predicted_scaled]])[0][0]
    return round(predicted_price, 2)




# 5. Utilitários

def get_feature_columns():
    return [
        "RSI", "MACD", "MACD_Signal", "SMA_50", "SMA_200", "Bollinger_Upper",
        "Bollinger_Lower", "ADX", "Stoch_K", "Stoch_D", "VWAP",
        "Doji", "Engulfing", "Hammer", "LSTM_PRED"
    ]

def generate_explanation(row, prediction):
    explanation = []
    if prediction == 1:
        explanation.append("🟢 O modelo prevê uma tendência de ALTA.")
    elif prediction == -1:
        explanation.append("🔴 O modelo prevê uma tendência de BAIXA.")
    else:
        explanation.append("⚪ Sinal neutro.")

    if row["RSI"] < 30:
        explanation.append("🔽 RSI abaixo de 30 indica sobrevenda.")
    elif row["RSI"] > 70:
        explanation.append("🔼 RSI acima de 70 indica sobrecompra.")

    if row["SMA_50"] > row["SMA_200"]:
        explanation.append("📈 SMA 50 acima da 200, tendência de alta.")
    else:
        explanation.append("📉 SMA 50 abaixo da 200, tendência de baixa.")

    if row["MACD"] > row["MACD_Signal"]:
        explanation.append("💹 MACD cruzando para cima, possível reversão positiva.")
    else:
        explanation.append("🔻 MACD abaixo da linha de sinal.")

    if row["Doji"] == 1:
        explanation.append("⚠️ Padrão de candle Doji detectado (possível reversão).")

    if row["Engulfing"] == 1:
        explanation.append("📊 Padrão de engolfo detectado (sinal forte de reversão).")

    return "\n".join(explanation)

def calculate_targets(current_price, direction, atr=0.02):
    if direction == 1:
        return {
            "TP1": round(current_price * (1 + atr * 0.5), 2),
            "TP2": round(current_price * (1 + atr * 1.0), 2),
            "SL": round(current_price * (1 - atr * 0.5), 2)
        }
    elif direction == -1:
        return {
            "TP1": round(current_price * (1 - atr * 0.5), 2),
            "TP2": round(current_price * (1 - atr * 1.0), 2),
            "SL": round(current_price * (1 + atr * 0.5), 2)
        }
    else:
        return {"TP1": None, "TP2": None, "SL": None}


def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "HTML"}
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        print("📨 Mensagem enviada com sucesso!")
    else:
        print(f"❌ Erro ao enviar mensagem: {response.status_code} - {response.text}")



def predict_next_closes(data, n_steps=5):
  from sklearn.ensemble import RandomForestRegressor

  df = data.copy().reset_index(drop=True)
  features = get_feature_columns()
  df.dropna(inplace=True)

  X = df[features]
  y = df["Close"].shift(-1).dropna()
  X = X.loc[y.index]

  if len(X) < 100:
      return [None] * n_steps

  model = RandomForestRegressor(n_estimators=200, max_depth=8, random_state=42)
  model.fit(X, y)

  last_row = df[features].iloc[-1].copy()
  preds = []

  for step in range(n_steps):
      X_input = pd.DataFrame([last_row], columns=features)
      next_close = model.predict(X_input)[0]
      preds.append(round(next_close, 2))

      # Simula avanço do mercado
      last_row["Close"] = next_close
      if "SMA_50" in last_row:
          last_row["SMA_50"] = last_row["SMA_50"] * 0.9 + next_close * 0.1
      if "SMA_200" in last_row:
          last_row["SMA_200"] = last_row["SMA_200"] * 0.95 + next_close * 0.05
      if "VWAP" in last_row:
          last_row["VWAP"] = last_row["VWAP"] * 0.95 + next_close * 0.05
      if "RSI" in last_row:
          last_row["RSI"] = min(100, max(0, last_row["RSI"] + np.random.normal(0, 0.5)))
      if "MACD" in last_row:
          last_row["MACD"] += np.random.normal(0, 0.3)
      if "MACD_Signal" in last_row:
          last_row["MACD_Signal"] += np.random.normal(0, 0.2)

      last_row = last_row[features]

  return preds





# 6. Execução

def run_analysis(
    selected_timeframes=None,
    plot_timeframes=["15m", "1h"],
    alert_timeframes=["15m", "1h", "1d"]
):
    if selected_timeframes is None:
        selected_timeframes = TIMEFRAMES

    results = []

    for asset in ASSETS:
        print(f"\n📊 Analisando {asset}...")
        models = {}
        lstm_models = {}
        data = {}

        try:
            for tf in selected_timeframes:
                interval = tf['interval']
                period = tf['period']
                data[interval] = calculate_indicators(get_stock_data(asset, interval, period))
                models[interval] = train_ml_model(data[interval], verbose=True)
                lstm_models[interval] = train_lstm_model(data[interval])
        except Exception as e:
            print(f"❌ Erro ao processar {asset}: {e}")
            continue

        if all(model is None for model in models.values()):
            print(f"⚠️ Nenhum modelo foi treinado para {asset}.")
            continue

        current_price = data.get("15m", data[list(data.keys())[0]])["Close"].iloc[-1]
        message = f"🔍 {asset}\n💰 Preço atual: ${current_price:.2f}\n"

        for tf in selected_timeframes:
            interval = tf['interval']
            model = models.get(interval)

            if model is None:
                message += f"\n⏱️ {interval}: Dados insuficientes para este timeframe.\n"
                continue

            latest_data = data[interval].iloc[-1]
            features = get_feature_columns()
            input_data = pd.DataFrame([latest_data[features]])
            prediction = model.predict(input_data)[0]
            proba = model.predict_proba(input_data)[0][np.where(model.classes_ == prediction)[0][0]]

            def safe_format(val): return f"{val:.2f}" if val is not None else "N/A"
            score = model.validation_score
            score_text = f"📈 Accuracy: {safe_format(score['accuracy'])} | Precision: {safe_format(score['precision'])} | Recall: {safe_format(score['recall'])}"

            targets = calculate_targets(current_price, prediction, tf['atr'])
            explanation = generate_explanation(latest_data, prediction)

            # Novo: Previsão com LSTM
            predicted_price_lstm = None
            if lstm_models.get(interval):
                try:
                    predicted_price_lstm = predict_with_lstm(lstm_models[interval], data[interval])
                except Exception as e:
                    print(f"[!] Erro na previsão LSTM: {e}")

            time_label = {"15m": "15 minutos", "1h": "1 hora", "1d": "Diário"}[interval]

            message += f"""
⏱️ {time_label}:
{'🟢 COMPRA' if prediction == 1 else ('🔴 VENDA' if prediction == -1 else '⚪ NEUTRO')} com {proba*100:.1f}% de confiança
🎯 TP1: ${targets['TP1']} | TP2: ${targets['TP2']} | SL: ${targets['SL']}
📌 Previsão: ${current_price * (1 + (tf['atr'] if prediction == 1 else (-tf['atr'] if prediction == -1 else 0))):.2f}
{score_text}
{explanation}
🔮 Preço previsto (LSTM): ${predicted_price_lstm if predicted_price_lstm else 'N/A'}
"""

            result = {
                "Asset": asset,
                "Timeframe": interval,
                "Date": latest_data.name,
                "Price": current_price,
                "Signal": prediction,
                "Confidence": round(proba, 4),
                "TP1": targets['TP1'],
                "TP2": targets['TP2'],
                "SL": targets['SL'],
                "Accuracy": score['accuracy'],
                "Precision": score['precision'],
                "Recall": score['recall'],
                "F1": score['f1'],
                "LSTM_Predicted": predicted_price_lstm
            }

            results.append(result)

            print(f"\n🔎 Verificando envio de alerta para {asset} | timeframe: {interval}")
            print(f"➡️ Confiança: {proba:.2f} | Alerta permitido? {interval in alert_timeframes}")
            print(f"➡️ Explicação:\n{explanation}")

            if interval in alert_timeframes:
                if proba > 0.60:
                    print("✅ Critério de confiança atendido (proba > 0.60)")
                if "MACD" in explanation and "SMA 50 acima" in explanation:
                    print("✅ Critério técnico atendido (MACD + SMA 50)")

                if proba > 0.60 or ("MACD" in explanation and "SMA 50 acima" in explanation):
                    send_telegram_message(message)
                    print("📨 Mensagem enviada para o Telegram!")
                else:
                    print("⛔ Alerta não enviado: confiança ou critério técnico insuficiente.")
            else:
                print("⛔ Timeframe não permitido para alerta.")

            if interval in plot_timeframes:
                try:
                    df = data[interval].copy().tail(50)
                    freq_map = {"15m": "15min", "1h": "h", "1d": "D"}
                    freq = freq_map.get(interval, "1min")
                    df["Datetime"] = (
                        df.index if isinstance(df.index, pd.DatetimeIndex)
                        else pd.date_range(end=pd.Timestamp.now(), periods=len(df), freq=freq)
                    )
                    plt.figure(figsize=(10, 4))
                    plt.plot(df["Datetime"], df["Close"], label="Preço")
                    if targets["TP1"]:
                        plt.axhline(targets['TP1'], color='green', linestyle='--', label='TP1')
                        plt.axhline(targets['TP2'], color='green', linestyle=':')
                        plt.axhline(targets['SL'], color='red', linestyle='--', label='SL')
                    plt.title(f"{asset} - {interval.upper()} - Previsão: {'COMPRA' if prediction == 1 else 'VENDA' if prediction == -1 else 'NEUTRO'}")
                    plt.legend()
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.grid()
                    plt.show()
                except Exception as e:
                    print(f"[!] Falha ao gerar gráfico: {e}")

    df_results = pd.DataFrame(results)
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"model_results_{timestamp}.csv"
    df_results.to_csv(filename, index=False)
    print(f"\n📁 Resultados salvos em: {filename}")

    win_trades = df_results[(df_results['Signal'] == 1) & (df_results['Confidence'] > 0.6)]
    if not win_trades.empty:
        avg_target = (win_trades['TP1'] - win_trades['Price']).mean()
        print(f"\n✅ Backtest TP1 médio: +${avg_target:.2f} por trade em sinais de compra com confiança > 60%")
    else:
        print("\n⚠️ Nenhum trade de compra acima de 60% de confiança para backtest.")


# Execução em loop (a cada 1 hora)
import time
from datetime import datetime

def is_time_to_run(interval):
    now = datetime.now()
    if interval == "15m":
        return now.minute % 15 == 0
    elif interval == "1h":
        return now.minute == 0
    elif interval == "1d":
        return now.hour == 8 and now.minute == 0
    return False

while True:
    now = datetime.now()
    print(f"\n⏰ Verificando timeframes - {now.strftime('%Y-%m-%d %H:%M:%S')}")

    for tf in TIMEFRAMES:
        interval = tf["interval"]
        if is_time_to_run(interval):
            print(f"\n🚀 Rodando análise para timeframe {interval}...")

            try:
                run_analysis(
                    selected_timeframes=[tf],
                    plot_timeframes=["1d"],        # Mude aqui se quiser ver gráfico de outros timeframes
                    alert_timeframes=["15m", "1h", "1d"]

                )
            except Exception as e:
                print(f"❌ Erro durante a análise de {interval}: {e}")
        else:
            print(f"⏳ Ainda não é hora para {interval}...")

    time.sleep(60)  # Verifica a cada 1 minuto




⏰ Verificando timeframes - 2025-03-27 19:56:01
⏳ Ainda não é hora para 15m...
⏳ Ainda não é hora para 1h...
⏳ Ainda não é hora para 1d...
